#### Econ 627: Assignment 5

##### <u> Question 2 </u>: Monte Carlo experiments with efficient instruments

In this exercise, using Monte Carlo simulations you will compare the small-sample performances of IV estimators based on efficient instruments with that of the 2SLS estimator.

######  Let us load the Libraries we need.

In [419]:
using Distributions, PrettyTables, Random, Parameters,DataFrames

a),  b) and c)

In [420]:
function generate_data(ρ,β,n)
      
    # IVs
    W = rand(Uniform(0,1), n)
    Z = -0.5 .*(W .< 0.2) .- 0.1 .* (0.2 .<= W .< 0.4) .+ 0.1 .* (0.4 .<= W .< 0.6) .+ (W .≥ 0.6)

    # errors
    Σ =[1.0 ρ; ρ 1.0;]
    mvnormal = MvNormal([0.0; 0.0], Σ)
    error = rand(mvnormal,n)'
    ϵ = error[:,1]
    V = error[:,2]
    U = (1 .+ Z).*ϵ

    # endogenous regressors Xi
    X = 4*Z.^2 + V
    
    # dependent variable Yi
    Y = β.*X .+ U

    return (Y = Y , X = X , Z = Z, U = U)

end


generate_data (generic function with 1 method)

In [421]:
@unpack X, Y, Z, U = generate_data(0.9, 0.15, 100);

d) Computations of the three IV estimators:

###### The 2SLS estimator using Zi as instruments.

In [422]:
# Function for estimation of the assymptotic variance Ω
function Ω(U,Z)
    n=length(U)
    zr = Z.*U
    omega = (zr' * zr)/n
     
    return omega
end

Ω (generic function with 1 method)

In [423]:
function β₂ₛₗₛ(Y,X,Z)
    
   n = length(Y)
   #Ωᵤ = Ω(U,Z) 
   Q = X'*Z/n
   W = inv(Z'*Z/n)
   PZ = Z*( (Z'*Z)\Z' )
   β = (X'*PZ*X)\(X'*PZ*Y)                                    #inv(Q*W*Q')*Q*W*(Z'*Y)
   Ωᵤ = Ω(Y-X*β,Z) 
   asy_var= ((Q'*W*Q)\(Q'*W*Ωᵤ*W*Q)/(Q'*W*Q))/n
   stderr = sqrt(asy_var)
   
   return β, stderr

end

β₂ₛₗₛ (generic function with 2 methods)

In [424]:
βᵢᵥ, stderrᵢᵥ  = β₂ₛₗₛ(Y,X,Z)

(0.08151880195811384, 0.08549805240959828)

###### The infeasible efficient IV estimator that uses g∗(Zi)

In [425]:
# Definition of the function g
 g(z) = mean(X .* (Z .== z)) / mean((Y-X*βᵢᵥ).^2 .* (Z .== z))

g (generic function with 1 method)

In [426]:
β_infeasible, stderr_inf = β₂ₛₗₛ(Y,X,g.(Z))

(0.14948661390596904, 0.0631568685499647)

###### The feasible version of the efficient IV estimator 

In [427]:
function βₒₗₛ(Y,X)
     
    Ω = inv(X'*X)
    Q = X'*Y;
    
    return Ω*Q
end

βₒₗₛ (generic function with 1 method)

In [428]:
D = [Z .== -0.5 Z .== -0.1 Z .== 0.1 Z .== 1 ]
π = βₒₗₛ(X,D)
πᵤ = βₒₗₛ(U.^2,D)

G  = (D*π)./(D*πᵤ)
β_feasible, stderr_fe = β₂ₛₗₛ(Y,X,G)

(0.15536572795816503, 0.06256193638504053)

e) Confidence intervals and associated asymptotic coverage probability 1 −α for different value of α

In [429]:
β = 0.15
for α in [0.1, 0.05, 0.01]
        lower_boundᵢᵥ = βᵢᵥ-quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        upper_boundᵢᵥ = βᵢᵥ+quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        lower_bound_inf = β_infeasible-quantile(Normal(0,1),1-α/2)*stderr_inf
        upper_bound_inf = β_infeasible+quantile(Normal(0,1),1-α/2)*stderr_inf
        lower_bound_fe = β_feasible-quantile(Normal(0,1),1-α/2)*stderr_fe
        upper_bound_fe = β_feasible+quantile(Normal(0,1),1-α/2)*stderr_fe

        table_data = ["βᵢᵥ" lower_boundᵢᵥ upper_boundᵢᵥ α lower_boundᵢᵥ<= β <= upper_boundᵢᵥ upper_boundᵢᵥ- lower_boundᵢᵥ;
                "β_infeasible" lower_bound_inf upper_bound_inf  α lower_bound_inf<= β <= upper_bound_inf -lower_bound_inf+upper_bound_inf 
                "β_feasible" lower_bound_fe upper_bound_fe   α lower_bound_fe<= β <= upper_bound_fe -lower_bound_fe+upper_bound_fe
        ]
        header=[" ","lower","upper", "α", "β in CI", "lenght of CI" ]
        pretty_table(table_data;header)
end

┌──────────────┬───────────┬──────────┬─────┬─────────┬──────────────┐
│              │     lower │    upper │   α │ β in CI │ lenght of CI │
├──────────────┼───────────┼──────────┼─────┼─────────┼──────────────┤
│          βᵢᵥ │ -0.059113 │ 0.222151 │ 0.1 │    true │     0.281264 │
│ β_infeasible │ 0.0456028 │  0.25337 │ 0.1 │    true │     0.207768 │
│   β_feasible │ 0.0524605 │ 0.258271 │ 0.1 │    true │      0.20581 │
└──────────────┴───────────┴──────────┴─────┴─────────┴──────────────┘
┌──────────────┬────────────┬──────────┬──────┬─────────┬──────────────┐
│              │      lower │    upper │    α │ β in CI │ lenght of CI │
├──────────────┼────────────┼──────────┼──────┼─────────┼──────────────┤
│          βᵢᵥ │ -0.0860543 │ 0.249092 │ 0.05 │    true │     0.335146 │
│ β_infeasible │  0.0257014 │ 0.273272 │ 0.05 │    true │      0.24757 │
│   β_feasible │  0.0327466 │ 0.277985 │ 0.05 │    true │     0.245238 │
└──────────────┴────────────┴──────────┴──────┴─────────┴────────


g) Repeat (a)-(f) 10,000 times. 


In [430]:
R=10^4
CI_length = zeros(3,3)
cov_prob = zeros(3,3)
prob_zero_in_CI = zeros(3,3)
levels =  [0.1, 0.05, 0.01]


for r=1:R

    @unpack X, Y, Z, U = generate_data(0.9, 0.15, 100);

    βᵢᵥ, stderrᵢᵥ  = β₂ₛₗₛ(Y,X,Z)
    β_infeasible, stderr_inf = β₂ₛₗₛ(Y,X,g.(Z))
    
    D = [Z .== -0.5 Z .== -0.1 Z .== 0.1 Z .== 1 ]
    π = βₒₗₛ(X,D)
    πᵤ = βₒₗₛ(U.^2,D)
    G  = (D*π)./(D*πᵤ)
    β_feasible, stderr_fe = β₂ₛₗₛ(Y,X,G)

    β = 0.15
    for i in 1:3
        
        α = levels[i]
        lower_boundᵢᵥ = βᵢᵥ-quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        upper_boundᵢᵥ = βᵢᵥ+quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        lower_bound_inf = β_infeasible-quantile(Normal(0,1),1-α/2)*stderr_inf
        upper_bound_inf = β_infeasible+quantile(Normal(0,1),1-α/2)*stderr_inf
        lower_bound_fe = β_feasible-quantile(Normal(0,1),1-α/2)*stderr_fe
        upper_bound_fe = β_feasible+quantile(Normal(0,1),1-α/2)*stderr_fe

        CI_length[1,i] += upper_boundᵢᵥ-lower_boundᵢᵥ
        CI_length[2,i] += upper_bound_inf-lower_bound_inf
        CI_length[3,i] += -lower_bound_fe+upper_bound_fe

        cov_prob[1,i] += (lower_boundᵢᵥ<= β <= upper_boundᵢᵥ)
        cov_prob[2,i] += (lower_bound_inf <= β <= upper_bound_inf)
        cov_prob[3,i] += (lower_bound_fe <= β <= upper_bound_fe)

        prob_zero_in_CI[1,i] += (0 < lower_boundᵢᵥ || 0> upper_boundᵢᵥ)
        prob_zero_in_CI[2,i] += (0 < lower_bound_inf || 0>upper_bound_inf)
        prob_zero_in_CI[3,i] += (0 < lower_bound_fe || 0>upper_bound_fe)
    end
end  


        


In [431]:
for i in 1:length(levels)
        table_data = ["βᵢᵥ" 1-levels[i] CI_length[1,i]/R cov_prob[1,i]/R prob_zero_in_CI[1,i]/R;
                "β_infeasible" 1-levels[i] CI_length[2,i]/R cov_prob[2,i]/R prob_zero_in_CI[2,i]/R;
                "β_feasible" 1-levels[i] CI_length[3,i]/R cov_prob[3,i]/R prob_zero_in_CI[3,i]/R;
                ]
                header=["estimator","1-α","lenght of CI","coverage prob","prob of signicance"]
                pretty_table(table_data;header)
end

┌──────────────┬─────┬──────────────┬───────────────┬────────────────────┐
│    estimator │ 1-α │ lenght of CI │ coverage prob │ prob of signicance │
├──────────────┼─────┼──────────────┼───────────────┼────────────────────┤
│          βᵢᵥ │ 0.9 │     0.277408 │        0.8879 │             0.5561 │
│ β_infeasible │ 0.9 │     0.205115 │        0.8748 │             0.7646 │
│   β_feasible │ 0.9 │     0.204345 │        0.8799 │             0.7847 │
└──────────────┴─────┴──────────────┴───────────────┴────────────────────┘
┌──────────────┬──────┬──────────────┬───────────────┬────────────────────┐
│    estimator │  1-α │ lenght of CI │ coverage prob │ prob of signicance │
├──────────────┼──────┼──────────────┼───────────────┼────────────────────┤
│          βᵢᵥ │ 0.95 │     0.330552 │        0.9402 │             0.4528 │
│ β_infeasible │ 0.95 │      0.24441 │        0.9269 │             0.6809 │
│   β_feasible │ 0.95 │     0.243492 │         0.931 │             0.7063 │
└──────────────┴───

h) Comment on whether the simulated coverage probabilities for each of the three
methods are close to the nominal levels of 1 −α. Does the result of Question 1
seem to hold in finite samples?

##### We observe that for each of the three methods the simulated coverage probalities are pretty close to the corresponding nominal levels 1−α.

i) 
##### The result shows that the infeseable estimator is the most powerful estimator since the lenght of the corresponding confidence interval is the smallest and the significance probabity is the greatest. Also, we observe slight discrepencies between the infeasible and feasible efficient IV methods in terms of the length and coverage probability.This is due to fact the estimation of the feasible estimator involved estimates of the conditional expections rather than their exact values.

j) Repeat the Monte Carlo experiment (as described in parts (a)-(g)) using the
sample size n = 400. Organize your results again in a table as in part (g).
Compare with the results for n = 100 and comment on the differences.

In [432]:
R=10^4
CI_length = zeros(3,3)
cov_prob = zeros(3,3)
prob_zero_in_CI = zeros(3,3)
levels =  [0.1, 0.05, 0.01]

3-element Vector{Float64}:
 0.1
 0.05
 0.01

In [433]:
for r=1:R

    @unpack X, Y, Z, U = generate_data(0.9, 0.15, 400);

    βᵢᵥ, stderrᵢᵥ  = β₂ₛₗₛ(Y,X,Z)
    β_infeasible, stderr_inf = β₂ₛₗₛ(Y,X,g.(Z))
    
    D = [Z .== -0.5 Z .== -0.1 Z .== 0.1 Z .== 1 ]
    π = βₒₗₛ(X,D)
    πᵤ = βₒₗₛ(U.^2,D)
    G  = (D*π)./(D*πᵤ)
    β_feasible, stderr_fe = β₂ₛₗₛ(Y,X,G)

    β = 0.15
    for i in 1:3
        
        α = levels[i]
        lower_boundᵢᵥ = βᵢᵥ-quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        upper_boundᵢᵥ = βᵢᵥ+quantile(Normal(0,1),1-α/2)*stderrᵢᵥ
        lower_bound_inf = β_infeasible-quantile(Normal(0,1),1-α/2)*stderr_inf
        upper_bound_inf = β_infeasible+quantile(Normal(0,1),1-α/2)*stderr_inf
        lower_bound_fe = β_feasible-quantile(Normal(0,1),1-α/2)*stderr_fe
        upper_bound_fe = β_feasible+quantile(Normal(0,1),1-α/2)*stderr_fe

        
        CI_length[1,i] += upper_boundᵢᵥ-lower_boundᵢᵥ
        CI_length[2,i] += upper_bound_inf-lower_bound_inf
        CI_length[3,i] += -lower_bound_fe+upper_bound_fe

        cov_prob[1,i] += (lower_boundᵢᵥ<= β <= upper_boundᵢᵥ)
        cov_prob[2,i] += (lower_bound_inf <= β <= upper_bound_inf)
        cov_prob[3,i] += (lower_bound_fe <= β <= upper_bound_fe)

        prob_zero_in_CI[1,i] += (0 < lower_boundᵢᵥ || 0> upper_boundᵢᵥ)
        prob_zero_in_CI[2,i] += (0 < lower_bound_inf || 0>upper_bound_inf)
        prob_zero_in_CI[3,i] += (0 < lower_bound_fe || 0>upper_bound_fe)
    end
end  

for i in 1:length(levels)
    table_data = ["βᵢᵥ" 1-levels[i] CI_length[1,i]/R cov_prob[1,i]/R prob_zero_in_CI[1,i]/R;
            "β_infeasible" 1-levels[i] CI_length[2,i]/R cov_prob[2,i]/R prob_zero_in_CI[2,i]/R;
            "β_feasible" 1-levels[i] CI_length[3,i]/R cov_prob[3,i]/R prob_zero_in_CI[3,i]/R;
            ]
            header=["estimator","1-α","lenght of CI","coverage prob","prob of signicance"]
            pretty_table(table_data;header)
end


┌──────────────┬─────┬──────────────┬───────────────┬────────────────────┐
│    estimator │ 1-α │ lenght of CI │ coverage prob │ prob of signicance │
├──────────────┼─────┼──────────────┼───────────────┼────────────────────┤
│          βᵢᵥ │ 0.9 │     0.139173 │        0.8909 │              0.955 │
│ β_infeasible │ 0.9 │     0.105112 │        0.8842 │             0.9951 │
│   β_feasible │ 0.9 │     0.105028 │        0.8857 │              0.996 │
└──────────────┴─────┴──────────────┴───────────────┴────────────────────┘
┌──────────────┬──────┬──────────────┬───────────────┬────────────────────┐
│    estimator │  1-α │ lenght of CI │ coverage prob │ prob of signicance │
├──────────────┼──────┼──────────────┼───────────────┼────────────────────┤
│          βᵢᵥ │ 0.95 │     0.165835 │        0.9409 │             0.9236 │
│ β_infeasible │ 0.95 │     0.125249 │        0.9393 │             0.9891 │
│   β_feasible │ 0.95 │     0.125149 │        0.9405 │             0.9901 │
└──────────────┴───

##### We see that when the sample size becomes large, the confident interval shrinks, The probability of significance increases,  and the infeasible estimator remains the most powerful with the smallest CI.